# Power Consumtion of India
   **TODOS**
   1. Get link for pdf
   2. Increment the date in link
   3. Download pdf and convert to csv to different folder
   4. Add date column to that csv
   5. Merage that csv with other perivious csv 
gpa_full.to_excel("gpa_full.xlsx", encoding='utf-8')   

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import datetime
import urllib.request
import tabula
import os
import glob
import csv
import time

In [2]:
# main PDF Download  link 
# https://npp.gov.in/public-reports/cea/daily/dgr/01-01-2018/dgr4-2018-01-01.pdf

# url = 'https://npp.gov.in/dgrReports'
# pdf_url = 'https://npp.gov.in/public-reports/cea/daily/dgr/01-01-2018/dgr4-2018-01-01.pdf'
# url_get = requests.get(url)
# soup = BeautifulSoup(url_get.content, 'html.parser')


# Download pdf from link and Convert to CSV

In [3]:
def get_csv(date):
        new_pdf_date = date.strftime("%d-%m-%Y")
        pdf_url = f'https://npp.gov.in/public-reports/cea/daily/dgr/{new_pdf_date}/dgr4-{date}.pdf' 
            
        urllib.request.urlretrieve(pdf_url, f'./repdf/{date}.pdf')

        if os.path.exists(f'./repdf/{date}.pdf'):  
            print(f'{date}.pdf Downloaded!')
            tabula.convert_into(f"./repdf/{date}.pdf", f"./recsv/{date}.csv", output_format="csv", pages='all')
            print(f'{date}.pdf Converted to CSV!')
        else:
            print('Filed To Download OR Convert')

# Clean CSV and Type Casting

In [4]:


def csv_clean(date):
    df = pd.read_csv(f'./recsv/{date}.csv')

    # Adjusting Columns and renaming
    df.rename(columns={'Forced\rMaintanence(MW)': 'Forced Maintanence(MW)', 'Monitored\rCap.(MW)': 'Monitored Cap.(MW)', 'Total Cap. Under\rMaintenace (MW)': 'Total Cap. Under Maintenace (MW)', 'Planned\rMaintanence (MW)': 'Planned Maintanence (MW)', 'Other Reasons\r(MW)': 'Other Reasons (MW)', 'Generation2019-20( MU )': 'Programme or Expected(MU)', 'Unnamed: 7': 'Actual(MU)', 'Unnamed: 8': 'Excess(+) / Shortfall (-)', 'Unnamed: 9': 'Deviation'}, inplace=True) 
        
    # Deleting unnecessary columns 
    df = df.drop(['Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'], axis=1)    
    
    # Deleting unnecessary Rows
    df = df[df["Power Station"].str.contains("Total") == False]
    df = df[df["Power Station"].str.contains('Northern') == False]
    df = df[df["Power Station"].str.contains('Southern') == False]
    df = df[df["Power Station"].str.contains('Eastern') == False]
    df = df[df["Power Station"].str.contains('North Eastern') == False]
    df = df[df["Power Station"].str.contains('Western') == False]
    df = df[df["Power Station"].str.contains('Power Station') == False]  
   
# Deleting row which has null valeus
#     df = df.dropna().reset_index(drop=True) 
        
    # Replacing NaN to 0
    df.fillna(0.0, inplace=True)
    
    # Adding Dates Column
    rows, columns = df.shape
    df.insert(0, 'Dates', [f'{date}'] * rows)   
    
    # Types Casting 

    # Date -> str -> datetime
    df['Dates'] = pd.to_datetime(df['Dates'])  
    
    # str -> float64
    
    df['Monitored Cap.(MW)']= df['Monitored Cap.(MW)'].str.replace(',', '')
    df['Total Cap. Under Maintenace (MW)']= df['Total Cap. Under Maintenace (MW)'].str.replace(',', '')
    df['Forced Maintanence(MW)']= df['Forced Maintanence(MW)'].str.replace(',', '')   
    df['Planned Maintanence (MW)']= df['Planned Maintanence (MW)'].str.replace(',', '')   
    df['Other Reasons (MW)'] = df['Other Reasons (MW)'].str.replace(',', '')        
    df['Deviation'] = df['Deviation'].str.replace(',', '')        
    
    df = df.astype({ 'Actual(MU)': 'float', 'Excess(+) / Shortfall (-)': 'float', 'Deviation': 'float', 'Programme or Expected(MU)': 'float', 'Other Reasons (MW)': 'float', 'Planned Maintanence (MW)': 'float', 'Monitored Cap.(MW)': 'float', 'Forced Maintanence(MW)': 'float', 'Total Cap. Under Maintenace (MW)': 'float'})

#     df['Monitored Cap.(MW)'] = df['Monitored Cap.(MW)'].apply(lambda x: float(x.split()[0].replace(',', '')))
#     df['Total Cap. Under Maintenace (MW)'] = df['Total Cap. Under Maintenace (MW)'].apply(lambda x: float(x.split()[0].replace(',', '')))
#     df['Planned Maintanence (MW)'] = df['Planned Maintanence (MW)'].apply(lambda x: float(x.split()[0].replace(',', '')))
#     df['Forced Maintanence(MW)'] = df['Forced Maintanence(MW)'].apply(lambda x: float(x.split()[0].replace(',', '')))
#     df['Programme or Expected(MU)'] = df['Programme or Expected(MU)'].apply(lambda x: float(x.split()[0].replace(',', '')))
#     df['Actual(MU)'] = df['Actual(MU)'].apply(lambda x: float(x.split()[0].replace(',', '')))
#     df['Excess(+) / Shortfall (-)'] = df['Excess(+) / Shortfall (-)'].apply(lambda x: float(x.split()[0].replace(',', '')))
#     df['Deviation'] = df['Deviation'].apply(lambda x: float(x.split()[0].replace(',', '')))

#     df['Other Reasons (MW)'] = df['Other Reasons (MW)'].apply(lambda x: float(x.split()[0].replace(',', '')))              
    
    #float -> int64
    df.fillna(0.0, inplace=True)
    
    df['Other Reasons (MW)'] = df['Other Reasons (MW)'].astype(int)
    df['Programme or Expected(MU)'] = df['Programme or Expected(MU)'].astype(int)
    df['Actual(MU)'] = df['Actual(MU)'].astype(int)
    return df     

# Convert df   ==>  xlsx   ==>   csv

In [5]:
def df_to_csv(date,df):
    excelfile = df.to_excel(f"./reexcel/{date}.xlsx", encoding='utf-8')
    data_xls = pd.read_excel(f'./reexcel/{date}.xlsx', index_col=None)
    data_xls.to_csv(f'./reCleanCsv/{date}.csv', encoding='utf-8', index=False)
    print(f'Clean csv exported: {date}.csv')

# Downloading files from 2017 to 2022
    - pdf -> csv
    - cleaing csv (removing unnecessary or repetitive data)
    - saving as csv

In [6]:
# i = 0
# date = datetime.date(2022,7,26)
# while date != datetime.date(2022,12,24):
#     get_csv(date)
#     if os.path.exists(f'./csv2/{date}.csv'):
#         df = csv_clean(date)
#     else:
#         print('Unable to Find Csv!!')
#     df_to_csv(date, df)
#     date += datetime.timedelta(days=1)

In [7]:
all_files = glob.glob('./CleanCsv/*.csv')

# Verifying data
    - checking numbers of rows and column which is smiliar in all file
    - if not then recreate that file and replace that with perivous         file (manuualy)

In [17]:
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

In [16]:
wrong_dfs = []
for df in li:
    row, column = df.shape
    if row < 170 or column < 12:
        date = datetime.datetime.strptime(df['Dates'][1], '%Y-%m-%d').date()
        wrong_dfs.append(date)
        get_csv(date)
        if os.path.exists(f"./recsv/{date}.csv"):
            dfcs = csv_clean(date)
        else:
            print('Unable to Find Csv!!')
        df_to_csv(date, dfcs)        

# Combining all data in one csv


In [18]:
df_main = pd.concat(li,axis=0)

# Again clean main csv( combined data file)

In [22]:
df_main = df_main.drop(['Generation2021-22( MU )', 'Generation2022-23( MU )'], axis=1)    

In [24]:
df_main = df_main.drop(['Unnamed: 0'], axis=1)    

In [31]:
df_main['Other Reasons (MW)'] = df_main['Other Reasons (MW)'].str.replace(',', '')        
df_main['Deviation'] = df_main['Deviation'].str.replace(',', '')  

In [49]:
df_main = df_main.dropna().reset_index(drop=True) 

# TypeCasting 

In [34]:
df_main = df_main.astype({'Other Reasons (MW)':'float', 'Deviation': 'float'})

In [43]:
df_main['Dates']= pd.to_datetime(df_main['Dates'])

In [41]:

df_main.fillna(0.0, inplace=True)

df_main['Other Reasons (MW)'] = df_main['Other Reasons (MW)'].astype(int)
df_main['Programme or Expected(MU)'] = df_main['Programme or Expected(MU)'].astype(int)
df_main['Actual(MU)'] = df_main['Actual(MU)'].astype(int)

# Exporting df as xlsx -> csv

In [54]:
df_main = df_main.set_index('Dates')

In [56]:
excelfile = df_main.to_excel(f"./CompleteData/PowerGeneration.xlsx", encoding='utf-8')
data_xls = pd.read_excel(f'./CompleteData/PowerGeneration.xlsx', index_col=None)
data_xls.to_csv(f'./CompleteData/CompleteData.csv', encoding='utf-8', index=False)

In [47]:
df_com = pd.read_csv('./CompleteData/CompleteData.csv')